# Read data

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.subplots import make_subplots

import os

# List all .txt files in the current directory
txt_files = [f for f in os.listdir('.') if f.endswith('.txt')]
print(txt_files)

fileName = txt_files[0]


data = pd.read_csv(fileName, delimiter=r"[ ,]+", engine="python")
df = pd.DataFrame(data)

In [ ]:
column_names = df.columns
df.columns

Index(['cycleCtr', 'time_InMs', 'forceRaw_InKg', 'forceFiltered_InKg',
       'forceVelocity_InKgPerSec', 'servoPos_InSteps', 'servoPosEsp_InSteps',
       'servoCurrent_InPercent', 'servoVoltage_InV'],
      dtype='object')

# Force vs. time

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Calculate mean and std deviation
mean_force = df['forceRaw_InKg'].mean()
std_force = df['forceRaw_InKg'].std()

# Create subplots: 1 row, 2 columns
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Force over Samples", f"Force Histogram<br>(mean = {mean_force:.5f} kg, std = {std_force:.5f} kg)"),
    horizontal_spacing=0.2
)

# Light blue color
light_blue = '#87CEFA'

# Left subplot: forceRaw_InKg over samples
fig.add_trace(
    go.Scatter(
        x=df.index,  # or df['time_InMs'] if desired
        y=df['forceRaw_InKg'],
        mode='lines',
        name='force (raw)',
        line=dict(color=light_blue)
    ),
    row=1, col=1
)

# Right subplot: histogram of forceRaw_InKg
fig.add_trace(
    go.Histogram(
        x=df['forceRaw_InKg'],
        name='force (raw)',
        histnorm='probability',
        marker_color=light_blue,
        showlegend=False
    ),
    row=1, col=2
)

# Layout and axes
fig.update_layout(
    height=500,
    width=1000,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.1,
        xanchor="center",
        x=0.5
    )
)

fig.update_xaxes(title_text="Sample index", row=1, col=1)
fig.update_yaxes(title_text="Force in kg", row=1, col=1)
fig.update_xaxes(title_text="Force in kg", row=1, col=2)
fig.update_yaxes(title_text="Frequency (relative)", row=1, col=2)

fig.show()

In [ ]:
fig = go.Figure()

from scipy.signal import butter, filtfilt

# --- Define sampling rate ---
# If your time is in milliseconds:
# Calculate fs from your time array if it's not constant
time_diff = np.diff(df['time_InMs']) / 1000.0  # convert ms to seconds
fs = 1 / np.mean(time_diff)  # effective sampling frequency in Hz

# --- Filter specifications ---
cutoff = 30  # Cutoff frequency in Hz
order = 4  # Order of the Butterworth filter

# --- Design low-pass Butterworth filter ---
b, a = butter(N=order, Wn=cutoff / (0.5 * fs), btype='low')

# --- Apply zero-phase filtering ---
df['forceFiltered_offline_InKg'] = filtfilt(b, a, df['forceRaw_InKg'])


df['forceDeviation'] = df['forceFiltered_offline_InKg'] - df['forceRaw_InKg']

# Define window duration in seconds
window_duration_s = 0.2  # 200 ms window

# Convert to number of samples
window_samples = int(window_duration_s * fs)

# Compute rolling std dev
df['forceDeviation_std'] = (df['forceDeviation']).rolling(window=window_samples, center=True).std()


# force related data

fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['forceRaw_InKg'], mode='lines', name='force (raw)', yaxis='y1')
)
fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['forceFiltered_offline_InKg'], mode='lines', name='force (non-causal filter)', yaxis='y1')
)
fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['forceFiltered_InKg'], mode='lines', name='force (filtered)', yaxis='y1')
)


fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['forceDeviation'], mode='lines', name='force (deviation from non-causal filter)', yaxis='y2')
)


# Step 2: Add symmetric (positive and negative) std traces to the plot
fig.add_trace(
    go.Scatter(
        x=df['time_InMs'],
        y=df['forceDeviation_std'],
        mode='lines',
        name='force deviation std (+)',
        line=dict(color='orange'),  # Choose your color
        yaxis='y2'
    )
)

fig.add_trace(
    go.Scatter(
        x=df['time_InMs'],
        y=-df['forceDeviation_std'],
        mode='lines',
        name='force deviation std (−)',
        line=dict(color='orange'),  # same color, optional dash
        yaxis='y2'
    )
)


# Layout for two axis
fig.update_layout(
    title="force with non-causal filter",
    xaxis=dict(title="time in ms"),
    yaxis=dict(
        title="Force in kg",
        #titlefont=dict(color="black"),
        #tickfont=dict(color="black")
    ),
    yaxis2=dict(
        title="Force deviation in kg",
        range=[-0.15, 0.15],  # Set limits for secondary y-axis
        #titlefont=dict(color="black"),
        #tickfont=dict(color="black"),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    legend=dict(
        orientation="h",  # Horizontale Anordnung
        yanchor="bottom",
        y=1.1,  # Leicht über den Plot verschieben
        xanchor="center",
        x=0.5
    )
)

fig.show()

# Force vs. position

In [ ]:
fig = go.Figure()

# force related data
fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['forceRaw_InKg'], mode='lines', name='force (raw)', yaxis='y1')
)
fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['forceFiltered_InKg'], mode='lines', name='force (filtered)', yaxis='y1')
)

# current related data
fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['servoPos_InSteps']+df['servoPosError_InSteps'], mode='lines', name='servo pos', yaxis='y2')
)

fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['servoPosEsp_InSteps'], mode='lines', name='ESP pos', yaxis='y2')
)


# Layout for two axis
fig.update_layout(
    title="force vs position",
    xaxis=dict(title="time in ms"),
    yaxis=dict(
        title="Force in kg",
        #titlefont=dict(color="black"),
        #tickfont=dict(color="black")
    ),
    yaxis2=dict(
        title="position in steps",
        #titlefont=dict(color="black"),
        #tickfont=dict(color="black"),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    legend=dict(
        orientation="h",  # Horizontale Anordnung
        yanchor="bottom",
        y=1.1,  # Leicht über den Plot verschieben
        xanchor="center",
        x=0.5
    )
)

fig.show()

# Estimated servo position lag

In [ ]:
fig = go.Figure()


# current related data
fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['servoPos_InSteps']+df['servoPosError_InSteps'], mode='lines', name='servo pos', yaxis='y1')
)

fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['servoPosEsp_InSteps'], mode='lines', name='ESP pos', yaxis='y1')
)

# fig.add_trace(
# go.Scatter(x=df['time_InMs'], y=df['servoPositionEstimated_stepperPos_i16'],
# mode='lines', name='ESP pos (at com)', yaxis='y1')
# )


fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['servoPosEstimated_InSteps'], mode='lines', name='estimated position error', yaxis='y2')
)




# Layout for two axis
fig.update_layout(
    title="position in steps",
    xaxis=dict(title="time in ms"),
    yaxis=dict(
        title="Position in steps",
        #titlefont=dict(color="black"),
        #tickfont=dict(color="black")
    ),
    yaxis2=dict(
        title="position error",
        #titlefont=dict(color="black"),
        #tickfont=dict(color="black"),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    legend=dict(
        orientation="h",  # Horizontale Anordnung
        yanchor="bottom",
        y=1.1,  # Leicht über den Plot verschieben
        xanchor="center",
        x=0.5
    )
)

fig.show()

# Servo current

In [ ]:
fig = go.Figure()

# force related data
fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['forceRaw_InKg'], mode='lines', name='force (raw)', yaxis='y1')
)
fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['forceFiltered_InKg'], mode='lines', name='force (filtered)', yaxis='y1')
)

# current related data
fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['servoCurrent_InPercent'], mode='lines', name='current', yaxis='y2')
)


# Layout for two axis
fig.update_layout(
    title="force vs current",
    xaxis=dict(title="time in ms"),
    yaxis=dict(
        title="Force in kg",
        #titlefont=dict(color="black"),
        #tickfont=dict(color="black")
    ),
    yaxis2=dict(
        title="current in %",
        #titlefont=dict(color="black"),
        #tickfont=dict(color="black"),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    legend=dict(
        orientation="h",  # Horizontale Anordnung
        yanchor="bottom",
        y=1.1,  # Leicht über den Plot verschieben
        xanchor="center",
        x=0.5
    )
)

fig.show()

In [ ]:
fig = go.Figure()


# force related data
fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['servoPos_InSteps'], mode='lines', name='servo pos', yaxis='y1')
)
fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['servoPosEsp_InSteps'], mode='lines', name='ESP pos', yaxis='y1')
)

# current related data
fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['servoCurrent_InPercent'], mode='lines', name='current', yaxis='y2')
)


# Layout for two axis
fig.update_layout(
    title="position vs current",
    xaxis=dict(title="time in ms"),
    yaxis=dict(
        title="position in steps",
        #titlefont=dict(color="black"),
        #tickfont=dict(color="black")
    ),
    yaxis2=dict(
        title="current in %",
        #titlefont=dict(color="black"),
        #tickfont=dict(color="black"),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    legend=dict(
        orientation="h",  # Horizontale Anordnung
        yanchor="bottom",
        y=1.1,  # Leicht über den Plot verschieben
        xanchor="center",
        x=0.5
    )
)

fig.show()

<class 'plotly.graph_objs._figure.Figure'>

# Servo voltage

In [ ]:
fig = go.Figure()


# force related data
#fig.add_trace(
#    go.Scatter(x=df['time_InMs'], y=df['servoPos_InSteps'], mode='lines', name='servo pos', yaxis='y1')
#)
fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['servoPosEsp_InSteps'], mode='lines', name='ESP pos', yaxis='y1')
)

# current related data
fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['servoVoltage_InV'], mode='lines', name='Voltage', yaxis='y2')
)


# Layout for two axis
fig.update_layout(
    title="position vs voltage",
    xaxis=dict(title="time in ms"),
    yaxis=dict(
        title="position in steps",
        #titlefont=dict(color="black"),
        #tickfont=dict(color="black")
    ),
    yaxis2=dict(
        title="voltage in V",
        #titlefont=dict(color="black"),
        #tickfont=dict(color="black"),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    legend=dict(
        orientation="h",  # Horizontale Anordnung
        yanchor="bottom",
        y=1.1,  # Leicht über den Plot verschieben
        xanchor="center",
        x=0.5
    )
)

fig.show()

In [ ]:
fig = go.Figure()


# force related data
fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['servoPos_InSteps'], mode='lines', name='servo pos', yaxis='y1')
)
fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['servoPosEsp_InSteps'], mode='lines', name='ESP pos', yaxis='y1')
)

# current related data
fig.add_trace(
    go.Scatter(x=df['time_InMs'], y=df['brakeResistorState_b'], mode='lines', name='brake resistor', yaxis='y2')
)


# Layout for two axis
fig.update_layout(
    title="position vs brake resistor",
    xaxis=dict(title="time in ms"),
    yaxis=dict(
        title="position in steps",
        #titlefont=dict(color="black"),
        #tickfont=dict(color="black")
    ),
    yaxis2=dict(
        title="brake resistor state",
        #titlefont=dict(color="black"),
        #tickfont=dict(color="black"),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    legend=dict(
        orientation="h",  # Horizontale Anordnung
        yanchor="bottom",
        y=1.1,  # Leicht über den Plot verschieben
        xanchor="center",
        x=0.5
    )
)

fig.show()